In [1]:
import numpy as np

# pydrostat
Muscular hydrostat simulation using constrained dynamics

This notebook demonstrates a simulation of a muscular hydrostat arm. 



# 1) Define a structure to simulate

The structure is the arm or tentacle or cell being simulated. It is any set of vertices connected by edges, faces, and cells (although it's not required to have any of those connections). This notebook will demonstrate with an arm made of cubic cells. 

In [2]:
from pydrostat.structure import structures

arm_height = 6
arm_builder = structures.CubicArmBuilder(arm_height)

arm_builder_2 = structures.CubicArmBuilder(arm_height)


# 2) Define an environment for the structure

An environment has food and obstacles. The food emmits a scent which diffuses through the environment around the obstacles (ie obstacles block the scent). This requires a simulation of the diffusion of the scent which approaches some steady state distribution of scent concentration.

Add this environment to the arm builder.

In [3]:
from pydrostat.environment import environment
from pydrostat.environment.obstacles import convex_polytope
from pydrostat.environment import food

food_loc = np.array([[-3, 1, 3]])
food_mag = 1
food_instance = food.Food(food_loc, food_mag)

rect_obst = convex_polytope.build_rectangular_obstacle([-2.5, -1, -1], [-1.5 ,9, 9])

env = environment.Environment(dim = 3, obstacles = [rect_obst], foods = [food_instance])
arm_builder.add_environment(env)

arm_builder_2.add_environment(env)


Loop: 50, Error: 1.000
Loop: 100, Error: 1.000
Loop: 150, Error: 0.337
Loop: 200, Error: 0.185
Loop: 250, Error: 0.115
Loop: 300, Error: 0.078
Loop: 350, Error: 0.056
Converged!


# 3) Define sensors and a controller for the structure

A structure takes sensory measurements of its environment and passes the sense data to a controller. Each type of controller may require a specific type of sensor data, or some combination of sensor data. 

In [4]:
from pydrostat.sensing.vertex_chemoceptors import VertexChemoceptors
from pydrostat.control.hand_tuned_gradient import HandTunedGradient
from pydrostat.control.hand_tuned_gradient import HandTunedGradient2

arm_builder.add_sensor(VertexChemoceptors())
arm_builder.add_controller(HandTunedGradient())

arm_builder_2.add_sensor(VertexChemoceptors())
arm_builder_2.add_controller(HandTunedGradient2())


# 4) Add constraints

Each constraint defines some sort of equality or inequality constraint that must be maintained throughout the simulation. This can be something like constant volume of a cell, minimum or maximum edge length, etc. 

In [5]:
from pydrostat.constraint.constant_volume import ConstantVolumeCommon
from pydrostat.constraint.fixed_vertex import FixedVertex
from pydrostat.constraint.planar_faces import PlanarFacesCommon
from pydrostat.constraint.edge_length import ClipLength

arm_builder.add_constraint(ConstantVolumeCommon())
arm_builder.add_constraint(FixedVertex([0,1,2,3]))
arm_builder.add_constraint(PlanarFacesCommon())
arm_builder.add_constraint(ClipLength(0.5, 2.5))

arm_builder_2.add_constraint(ConstantVolumeCommon())
arm_builder_2.add_constraint(FixedVertex([0,1,2,3]))
arm_builder_2.add_constraint(PlanarFacesCommon())
arm_builder_2.add_constraint(ClipLength(0.5, 2.5))

# 5) Build the arm

In [6]:
arm = arm_builder.construct_arm()
arm_2 = arm_builder_2.construct_arm()

In [7]:
# arm.apply_external_forces(4, np.array([-1,-1,0])*2)
# arm.apply_external_forces(4, np.array([0,0,-1])*0.5)

In [ ]:
from pydrostat.render.display import DisplayStructure

render_display = DisplayStructure(env, [arm], dt=1/60)
render_display.main_loop(simulating=False)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Actual FPS: 0.00 | Loop Period (ms): 16.82 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.82 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.06 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.05 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.83 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.46 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 15.86 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.63 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.91 | Sim Time (ms): 0.00
Actual FPS: 0.00 | Loop Period (ms): 16.42 | Sim Time (ms): 0.00
Actual FPS: 61.73 | Loop Period (ms): 15.80 | Sim Time (ms): 0.00
Actual FPS: 61.73 | Loop Period (ms): 16.15 | Sim Time (ms): 0.00
Actual FPS: 61.73 | Loop Period (ms): 15.77 | Sim Time (ms): 0.00
Actual FPS: 61.73 | Loop Period (ms): 1

/home/alex/code/muscular_hydrostat/src/pydrostat/constraint/planar_faces.py:288: RuntimeWarning: divide by zero encountered in divide
  deigval_difdt[:] = -np.divide(


Actual FPS: 41.49 | Loop Period (ms): 22.50 | Sim Time (ms): 16.87
Actual FPS: 41.49 | Loop Period (ms): 19.92 | Sim Time (ms): 16.44
Actual FPS: 41.49 | Loop Period (ms): 25.83 | Sim Time (ms): 22.35
Actual FPS: 41.49 | Loop Period (ms): 22.32 | Sim Time (ms): 18.93
Actual FPS: 42.92 | Loop Period (ms): 22.73 | Sim Time (ms): 16.66
Actual FPS: 42.92 | Loop Period (ms): 24.27 | Sim Time (ms): 20.46
Actual FPS: 42.92 | Loop Period (ms): 20.42 | Sim Time (ms): 16.95
Actual FPS: 42.92 | Loop Period (ms): 20.30 | Sim Time (ms): 15.52
Actual FPS: 42.92 | Loop Period (ms): 20.36 | Sim Time (ms): 16.37
Actual FPS: 42.92 | Loop Period (ms): 18.70 | Sim Time (ms): 15.24
Actual FPS: 42.92 | Loop Period (ms): 20.77 | Sim Time (ms): 17.16
Actual FPS: 42.92 | Loop Period (ms): 19.59 | Sim Time (ms): 16.04
Actual FPS: 42.92 | Loop Period (ms): 20.90 | Sim Time (ms): 15.66
Actual FPS: 42.92 | Loop Period (ms): 20.22 | Sim Time (ms): 16.33
Actual FPS: 47.62 | Loop Period (ms): 21.77 | Sim Time (ms): 1

SystemExit: 

/home/alex/code/muscular_hydrostat/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
